## 예술의전당 콘서트홀 가격 모델

### 1. 라이브러리

In [199]:
# base
import pandas as pd
import numpy as np
import re

# files
from glob import glob
import warnings
from tqdm import tqdm

# visualization
import matplotlib.pyplot as plt
import koreanize_matplotlib
import plotly.express as px
import seaborn as sns
import matplotlib.colors as mcolors
from matplotlib.cm import ScalarMappable

# M.L
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, silhouette_score
from kneed import KneeLocator

In [200]:
# settings
%matplotlib inline
pd.set_option("display.max_columns", 100)
pd.set_option('display.max_rows', 100)
warnings.simplefilter("ignore")


### 2. 데이터셋 로드

In [205]:
# 기존의 제공된 데이터프레임에 빈좌석, 좌표, 무대와의 관계가 추가된 데이터프레임
file_name = "빈좌석_포함_클래식_데이터.csv"
df = pd.read_csv(glob(file_name)[0], low_memory=False)

# 필요없는 컬럼 삭제
df = df.drop(['membership_type_1', 'membership_type_2', 'membership_type_3', 
              'membership_type_4', 'membership_type_5', 'membership_type_6',], axis=1)

print(f"""Found.. {len(glob(file_name))} file(s) : {glob(file_name)}
Reading.. {glob(file_name)[-1]}
df.shape : {df.shape}""")


Found.. 1 file(s) : ['빈좌석_포함_클래식_데이터.csv']
Reading.. 빈좌석_포함_클래식_데이터.csv
df.shape : (405810, 34)


### 3. 클래스/함수 : 원가격, 원등급 추정

In [206]:
class Performance:
    instance_cnt = 0

    def __init__(self, df):
        Performance.instance_cnt += 1
        self.instance_cnt = Performance.instance_cnt
        self.df = df
        self.perform_time = df['전체공연시간'].iloc[0]
        self.get_original_price()
    

    def get_original_price(self):
        """
        discount_type에서 할인율을 추출하고 역산해서 '할인전금액'을 df에 컬럼으로 추가하는 함수
        """
        self.df['할인율'] = self.df['discount_type'].str.extract('(\d+)%')
        self.df['할인율'] = self.df['할인율'].fillna(0).astype(int) / 100
        self.df['할인전가격'] = (self.df['price'].fillna(0) // (1 - self.df['할인율'])).round(-2).astype(int)
        self.df['원가격추정'] = self.df['할인전가격'].copy()
        self.priced_seat = self.df[self.df['원가격추정'] > 0]
        self.unpriced_seat = self.df[self.df['원가격추정'] == 0]
        self.priced_rate = round((self.priced_seat.shape[0] / self.df.shape[0]), 3)
        self.booked_rate = round(self.df['예매여부'].mean(), 3)


    def get_best_n_neighbors(self):
        """
        knn가격 추정 모델의 적절한 n_neighbors값을 찾는 함수
        """
        X, y = self.priced_seat[['X', 'Y', 'Z']], self.priced_seat['원가격추정']
        cv_scores = []
        for n in range(1, min(50, (X.shape[0]*9//10)) + 1):
            model = KNeighborsRegressor(n_neighbors=n, weights='distance', p=2)
            scores = cross_val_score(model, X, y, cv=10, scoring='neg_mean_squared_error')
            cv_scores.append(scores.mean())
            self.best_n_neighbors = [i for i in range(1, 51)][np.argmax(cv_scores)]
            self.knn_mse = max(cv_scores) * -1


    def estimate_price(self):
        """
        knn모델에 추정된 n_neighbors값을 적용해서 판매되지 않은 티켓의 가격을 추정하는 함수
        """
        # 가격을 하나도 알 수 없는 경우 종료
        if self.priced_rate == 0:
            self.best_n_neighbors = 1
            self.knn_mse = 0
            self.mean_price = self.max_price = self.min_price = 0
            return
        
        # priced_rate != 0 이면 n_neighbors값 찾기
        self.get_best_n_neighbors()

        # 찾은 n_neighbors 값으로 knn 가격추정
        X, y = self.priced_seat[['X', 'Y', 'Z']], self.priced_seat['원가격추정']
        model = KNeighborsRegressor(n_neighbors=self.best_n_neighbors, weights='distance', p=2)
        model.fit(X, y)
        y_pred = model.predict(self.unpriced_seat[['X', 'Y', 'Z']]).round(-2)
        self.df.loc[self.df['원가격추정']==0, '원가격추정'] = y_pred
        
        self.mean_price = self.df['원가격추정'].mean().round(2)
        self.max_price = int(self.df['원가격추정'].max())
        self.min_price = int(self.df['원가격추정'].min())

    def estimate_cluster_kmeans(self):
        # 중복되는 값이 없도록 난수를 더해서 노이즈 만들기
        self.df['rand'] = np.random.rand(self.df.shape[0])
        self.df['원가격추정_rand'] = self.df['원가격추정'] + self.df['rand']

        # 군집화 모델 생성 (K-means)
        X = self.df[['원가격추정_rand']]
        inertia = []
        k_range = range(1, 11)
        for k in k_range:
            kmeans = KMeans(n_clusters=k, random_state=42, n_init='auto')
            kmeans.fit(X)
            inertia.append(kmeans.inertia_)
        
        # 적절한 K값 찾기 (elbow point)        
        kneedle = KneeLocator(k_range[1:], inertia[1:], curve='convex', direction='decreasing')
        self.best_k = kneedle.elbow

        # 찾은 K값을 적용해서 원등급 추정하기
        kmeans = KMeans(n_clusters=self.best_k, random_state=42, n_init='auto')
        self.df['원등급추정'] = kmeans.fit_predict(X)
        cluster_means = self.df.groupby('원등급추정')['원가격추정'].mean().reset_index()
        cluster_means = cluster_means.sort_values(by='원가격추정')
        grade_mapping = {grade: idx for idx, grade in enumerate(cluster_means['원등급추정'])}
        self.df = self.df.drop(['원가격추정_rand', 'rand'], axis=1)

        # 원등급, 등급별 가격의 비율, 군집분석의 실루엣점수
        self.df['원등급추정'] = self.df['원등급추정'].map(grade_mapping) + 1
        self.seat_price_ratio = list((cluster_means['원가격추정'] / min(cluster_means['원가격추정'])).round(2))
        self.silhouette_score = silhouette_score(self.df[['원가격추정']], self.df['원등급추정'])


    def px3dscatter(self, col_name):
        # 결과 시각화
        fig = px.scatter_3d(self.df, x='X', y='Y', z='Z', color=col_name, 
                            hover_name='seat', hover_data=['예매여부', '원가격추정', '원등급추정'],
                            width=800, height=600)
        fig.update_traces(marker={'size': 1})
        fig.show()

### 4. 클래스/함수 적용

In [207]:
# 전체 데이터를 공연시간을 기준으로 공연별 분할
공연시간_list = sorted(df['전체공연시간'].unique())
공연별_df_list = [df[df['전체공연시간'] == 공연시간] for 공연시간 in 공연시간_list]

In [208]:
# 공연별 군집분석결과
instance_cnt_list = []
perform_time_list = []
priced_rate_list = []
booked_rate_list = []
best_n_neighbors_list = []
knn_mse_list = []
best_k_list = []
silhouette_score_list = []
mean_price_list = []
max_price_list = []
min_price_list = []
seat_price_ratio_list = []
df_list = []

for 공연별_df in tqdm(공연별_df_list):
    p = Performance(공연별_df)
    p.estimate_price() # knn => 원가격 추정
    p.estimate_cluster_kmeans() # k-means => 원등급 추정

    instance_cnt_list.append(p.instance_cnt)
    perform_time_list.append(p.perform_time)
    priced_rate_list.append(p.priced_rate)
    booked_rate_list.append(p.booked_rate)
    best_n_neighbors_list.append(p.best_n_neighbors)
    knn_mse_list.append(p.knn_mse)
    best_k_list.append(p.best_k)
    silhouette_score_list.append(p.silhouette_score)
    mean_price_list.append(p.mean_price)
    max_price_list.append(p.max_price)
    min_price_list.append(p.min_price)
    seat_price_ratio_list.append(p.seat_price_ratio)
    df_list.append(p.df)

# 군집결과 데이터프레임으로 합치기
new_df = pd.concat(df_list, axis=0)
공연정보 = pd.DataFrame({
    '공연시간' : perform_time_list,
    '금액명시비율' : priced_rate_list,
    '예약율' : booked_rate_list,
    'knn_n_neighbors' : best_n_neighbors_list,
    'knn_mse' : knn_mse_list,
    'kmeans_군집수' : best_k_list,
    'kmeans_실루엣' : silhouette_score_list,
    '평균가격' : mean_price_list,
    '최소가격' : min_price_list,
    '최대가격' : max_price_list,
    '군집별가격비율' : seat_price_ratio_list
})

100%|██████████| 162/162 [02:15<00:00,  1.20it/s]


총 162개의 클래식공연 중..

11개의 공연은 가격이 전혀 명시되지 않았음.  
해당 공연의 데이터에서는 원가격, 원등급 추정이 불가함.  
이 중 일부는 모두 초대석으로 예매순서 또한 무의미함.  
  
앞으로의 EDA를 위해 151개의 공연 데이터만 사용하기로 함.

In [219]:
# 공연별 (추정된) 가격, 등급 정보 요약
공연정보_151 = 공연정보.loc[공연정보['금액명시비율']!=0].reset_index().drop('index', axis=1)
공연정보_151.to_csv('공연정보요약_151공연.csv', index=False)

In [211]:
# 전체공연좌석별 예매 데이터
공연151개_list = []
for 공연시간 in list(공연정보.loc[공연정보['금액명시비율']!=0, '공연시간']):
    공연151개_list.append(new_df[new_df['전체공연시간'] == 공연시간])

new_df_151 = pd.concat(공연151개_list, axis=0, ignore_index=True)
new_df_151.to_parquet('원가격및등급추정_151공연_데이터.parquet', index=False)